In [1]:
# import libraries

import openai 
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import pinecone
from langchain_community.llms import OpenAI

/home/muhammed-shafeeh/AI_ML/GenAI_Project/End_to_end_project_llm_with_pinecone_db/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

import os

In [3]:
def read_pdf(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [4]:
doc = read_pdf('/home/muhammed-shafeeh/AI_ML/GenAI_Project/End_to_end_project_llm_with_pinecone_db/data')
doc

[Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2023-02-01T05:28:04+05:30', 'moddate': '2023-02-01T08:28:21+05:30', 'title': '', 'source': '/home/muhammed-shafeeh/AI_ML/GenAI_Project/End_to_end_project_llm_with_pinecone_db/data/budget_speech.pdf', 'total_pages': 58, 'page': 0, 'page_label': '1'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023'),
 Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2023-02-01T05:28:04+05:30', 'moddate': '2023-02-01T08:28:21+05:30', 'title': '', 'source': '/home/muhammed-shafeeh/AI_ML/GenAI_Project/End_to_end_project_llm_with_pinecone_db/data/budget_speech.pdf', 'total_pages': 58, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2

In [5]:
def chunk_data(documents, chunk_size=800, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_documents(documents)
    return chunks

In [6]:
chunks = chunk_data(doc)

In [7]:
chunks

[Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2023-02-01T05:28:04+05:30', 'moddate': '2023-02-01T08:28:21+05:30', 'title': '', 'source': '/home/muhammed-shafeeh/AI_ML/GenAI_Project/End_to_end_project_llm_with_pinecone_db/data/budget_speech.pdf', 'total_pages': 58, 'page': 0, 'page_label': '1'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023'),
 Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2023-02-01T05:28:04+05:30', 'moddate': '2023-02-01T08:28:21+05:30', 'title': '', 'source': '/home/muhammed-shafeeh/AI_ML/GenAI_Project/End_to_end_project_llm_with_pinecone_db/data/budget_speech.pdf', 'total_pages': 58, 'page': 2, 'page_label': '3'}, page_content='CONTENTS \nPART-A \n Page No. \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vi

In [8]:
openai_api_key = os.getenv('OPENAI_API_KEY')

In [9]:
## embedding 
embeddings = OpenAIEmbeddings(api_key=openai_api_key)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7c64401a5810>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7c6432f29b90>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [10]:
vector_stores = embeddings.embed_query('how are you')
len(vector_stores)


1536

In [11]:
## vector search in pinecone
pinecone_api_key = os.getenv('PINECONE_API_KEY')


In [12]:
#print(pinecone_api_key)

In [13]:

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)


In [26]:
index_name = "langchainvector"

pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '4b464e21de4b5e5008329c188931cd15', 'Date': 'Sat, 15 Mar 2025 14:27:15 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [27]:
from langchain_pinecone import PineconeVectorStore

In [28]:

docsearch = PineconeVectorStore(
    index=pc.Index(index_name),  
    embedding=embeddings,
)
docsearch.add_documents(chunks) 

['80664865-a256-49fb-9e15-3a5f99ef433d',
 '1194e15f-dcf9-43e2-bcf7-7205a2ce8477',
 '01b1c56e-83a4-42e9-b015-9abcff98d0ef',
 '418893f7-cb00-49f5-bf3c-e654ae5f25b5',
 '22391494-ae6d-480c-a855-06538d7bf4a2',
 '32a66b59-15cb-4b34-b41b-dadbc10824a4',
 '7d0ecc53-0c6e-4fde-925d-56f154f335a9',
 '3533e675-f99b-4bdc-82b0-d7f53393645f',
 '38bf0d75-70d4-4aea-98f4-c0bfbe20e94c',
 '102fa6e8-2d5a-419c-aced-bceed26650f2',
 '292f3cce-e68d-492e-8fb5-463813291f08',
 '230e0690-2f89-44d9-bb3f-2982d742540e',
 '8d255696-c723-4aef-8330-95db48884e9d',
 '4cbd2859-d5c1-4362-bea7-f4ee2773527d',
 'bc1fe1cb-603d-42a1-8d88-419323450ff5',
 'b85bbe23-63f2-42d1-91b0-efa812faf378',
 '3d07d6c4-8e19-4284-aa59-c45f14f6aae5',
 '54696222-88a0-4e14-9886-b19ba9749959',
 '00b02b7d-7c92-410a-9b1d-cd596e82449e',
 '271bee35-8a2a-45a4-bc16-f9cddec37d9c',
 '7c8e2601-f664-48b9-80b6-dab8d28d2a2c',
 '9a537cb9-7ee1-4f65-990b-b6d215437790',
 '35d20bec-0cfe-4071-aa62-5fda1fb63bb6',
 '251bb273-0f6d-42bc-adef-707f814c55aa',
 '62095937-2e01-

In [29]:
# cosine similarity retrieval
def rerieve_query(query,k=2):
    matching_results = pc.index.similarity_search(query,k=k)
    return matching_results

In [30]:
docsearch

In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [32]:
retriever_doc = retriever.invoke("What is a gdp of inda?")
print(retriever_doc[0].page_content)

expenditure of about ` 2 lakh crore will be borne by the Central 
Government. 
G20 Presidency: Steering the global agenda through challenges 
6. In these times of global challenges, the G20 Presidency gives us a 
unique opportunity to strengthen India’s role in the world economic order. 
With the theme of ‘ Vasudhaiva Kutumbakam’ , we are steering an 
ambitious, people-centric agenda to address global challenges, and to 
facilitate sustainable economic development.  
Achievements since 2014: Leaving no one behind 
7. The government’s efforts since 2014 have ensured for all citizens a 
better quality of living and a life of dignity. The per capita income has more 
than doubled to ` 1.97 lakh.   
8. In these nine years, the Indian economy has increased in size from


In [21]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.3,max_tokens=500)

In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [25]:
question_answer_chain = create_stuff_documents_chain(
    llm,
    prompt=prompt
)
rag_chain = create_retrieval_chain(
    retriever,
    question_answer_chain,
)

NameError: name 'retriever' is not defined

In [24]:
response = rag_chain.invoke({"input": "What is gdp of india?"})
print(response["answer"])

NameError: name 'rag_chain' is not defined